In [195]:
import pandas as pd
import string
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import unicodedata
from sklearn.linear_model import RidgeCV
import numpy as np
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import shap
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import models, layers
import xgboost as xgb
import tensorflow as tf
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
from sdv.metadata import SingleTableMetadata
import os
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer

In [174]:
X_train = pd.read_csv('/Users/emilfalk/Desktop/X_train.csv')
X_test = pd.read_csv('/Users/emilfalk/Desktop/X_test.csv')
y_train = pd.read_csv('/Users/emilfalk/Desktop/y_train.csv')
y_test = pd.read_csv('/Users/emilfalk/Desktop/y_test.csv')

In [175]:
X_train = X_train.drop(columns=['Player']) 
X_test = X_test.drop(columns=['Player']) 

In [176]:
X_train = X_train.drop(['years_with_team', 'draft_pos', 'all_star', 'num_teams', 'ig_followers', 'resigned', 'Agent'], axis=1)
X_test = X_test.drop(['years_with_team', 'draft_pos', 'all_star', 'num_teams', 'ig_followers', 'resigned', 'Agent'], axis=1)

In [177]:
from sdv.single_table import CTGANSynthesizer

df_train = X_train.copy()
df_train['salary'] = y_train.values.ravel() 

metadata = Metadata.detect_from_dataframe(data=df_train)

synthesizer = CTGANSynthesizer(metadata, epochs=4500)
synthesizer.fit(df_train)

synthetic_data = synthesizer.sample(num_rows=7000)

X_synth = synthetic_data.drop(columns=['salary'])
y_synth = synthetic_data[['salary']].copy()  
y_synth.columns = ['Salary']   


X_aug = pd.concat([X_train, X_synth], ignore_index=True)
y_aug = pd.concat([y_train, y_synth], ignore_index=True)

"""X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_aug, y_aug, test_size=0.2, random_state=42
)"""

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sdv/single_table/base.py:123: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



'X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(\n    X_aug, y_aug, test_size=0.2, random_state=42\n)'

In [178]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_aug) 
X_test_scaled = scaler.transform(X_test)

In [179]:
X_train_scaled_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

## Ridge Regression

In [180]:
ridge = Ridge(random_state=42)

param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

grid_search_basketball = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best Ridge alpha (Basketball Features): {grid_search_basketball.best_params_['alpha']}")

Best Ridge alpha (Basketball Features): 100.0


## Lasso Regression

In [181]:
lasso = Lasso(random_state=42)
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}


grid_search_basketball = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best Lasso alpha (Basketball Features): {grid_search_basketball.best_params_['alpha']}")

Best Lasso alpha (Basketball Features): 100.0


## Decision Tree

In [182]:
dt = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10]
}

grid_search_basketball = GridSearchCV(dt, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best Decision Tree params (Basketball Features): {grid_search_basketball.best_params_}")

Best Decision Tree params (Basketball Features): {'max_depth': 5, 'min_samples_split': 2}


## Random Forest

In [183]:
rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None]
}

grid_search_basketball = GridSearchCV(rf, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best Random Forest params (Basketball Features): {grid_search_basketball.best_params_}")

Best Random Forest params (Basketball Features): {'max_depth': 10, 'n_estimators': 200}


## XGBoost

In [184]:
xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1]
}

grid_search_basketball = GridSearchCV(xgb, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best XGBoost params (Basketball Features): {grid_search_basketball.best_params_}")

Best XGBoost params (Basketball Features): {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


## Multilayer Perceptron

In [185]:
mlp = MLPRegressor(random_state=42, max_iter=300)

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'learning_rate_init': [0.001, 0.01]
}

grid_search_basketball = GridSearchCV(mlp, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_basketball.fit(X_train_scaled, y_aug.values.ravel())
print(f"Best MLP params (Basketball Features): {grid_search_basketball.best_params_}")

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reac

Best MLP params (Basketball Features): {'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01}


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.



In [187]:
models = [
    {
        'name': 'Linear Regression',
        'model_basketball': LinearRegression()
    },
    {
        'name': 'Ridge Regression',
        'model_basketball': Ridge(alpha=100.0)
    },
    {
        'name': 'Lasso Regression',
        'model_basketball': Lasso(alpha=100.0)
    },
    {
        'name': 'Decision Tree',
        'model_basketball': DecisionTreeRegressor(max_depth=5, min_samples_split=2)
    },
    {
        'name': 'Random Forest',
        'model_basketball': RandomForestRegressor(max_depth=10, n_estimators=200)
    },
    {
        'name': 'XGBoost',
        'model_basketball': XGBRegressor(learning_rate=0.1, max_depth=3, n_estimators=100)
    },
    {
        'name': 'MLP',
        'model_basketball': MLPRegressor(hidden_layer_sizes=(50, 50), learning_rate_init=0.01, max_iter=300)
    }
]

cv_folds = 5
kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
mae_scale = 1_000_000

def evaluate_model(model, X_train, X_test, y_train, y_test, feature_set, model_name):
    if X_train.ndim > 2:
        X_train = X_train.reshape(X_train.shape[0], -1)
    if X_test.ndim > 2:
        X_test = X_test.reshape(X_test.shape[0], -1)

    model.fit(X_train, y_train.values.ravel())

    y_test_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    mae_folds = []
    for train_idx, val_idx in kf.split(X_train):
        X_train_fold = X_train[train_idx]
        X_val_fold = X_train[val_idx]
        y_train_fold = y_train.iloc[train_idx] if isinstance(y_train, pd.DataFrame) else y_train[train_idx]
        y_val_fold = y_train.iloc[val_idx] if isinstance(y_train, pd.DataFrame) else y_train[val_idx]

        model.fit(X_train_fold, y_train_fold.values.ravel() if isinstance(y_train_fold, pd.DataFrame) else y_train_fold)
        
        y_pred_fold = model.predict(X_val_fold)
        mae_folds.append(mean_absolute_error(y_val_fold, y_pred_fold))
    
    cv_mae_std = np.std(mae_folds, ddof=1) / mae_scale
    
    return model, mae_folds, y_test_pred, test_mae, test_r2

for model_info in models:
    name = model_info['name']
    model_basketball = model_info['model_basketball']
    
    _, mae_basketball_folds, y_test_pred_basketball, test_mae, test_r2 = evaluate_model(
        model_basketball, 
        X_train_scaled, 
        X_test_scaled, 
        y_aug, 
        y_test, 
        'Basketball Features', 
        name
    )
    
    print(f"\nDiagnostics for {name}:")
    print(f"Basketball Features MAE Folds: {[f'{mae:.2f}' for mae in mae_basketball_folds]}")
    print(f"Basketball Features MAE Mean: {np.mean(mae_basketball_folds):.2f}")
    print(f"Basketball Features MAE Std: {np.std(mae_basketball_folds, ddof=1):.2f}")
    print(f"Basketball Features MAE: {test_mae:.4f}")
    print(f"Basketball Features R2: {test_r2:.4f}")


Diagnostics for Linear Regression:
Basketball Features MAE Folds: ['4530240.20', '4632500.82', '4460727.12', '4636020.62', '4740389.54']
Basketball Features MAE Mean: 4599975.66
Basketball Features MAE Std: 107617.81
Basketball Features MAE: 4310212.3404
Basketball Features R2: 0.6393

Diagnostics for Ridge Regression:
Basketball Features MAE Folds: ['4536347.27', '4635419.32', '4465140.11', '4645482.42', '4741717.48']
Basketball Features MAE Mean: 4604821.32
Basketball Features MAE Std: 106688.40
Basketball Features MAE: 4305512.5470
Basketball Features R2: 0.6407

Diagnostics for Lasso Regression:
Basketball Features MAE Folds: ['4530256.80', '4632473.89', '4460708.44', '4636057.96', '4740371.42']
Basketball Features MAE Mean: 4599973.70
Basketball Features MAE Std: 107616.35
Basketball Features MAE: 4310204.7896
Basketball Features R2: 0.6393

Diagnostics for Decision Tree:
Basketball Features MAE Folds: ['4357182.56', '4402120.55', '4201973.23', '4335299.72', '4530140.54']
Basketb

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged ye


Diagnostics for MLP:
Basketball Features MAE Folds: ['4050907.90', '4219692.76', '3926304.70', '4113868.59', '4212231.95']
Basketball Features MAE Mean: 4104601.18
Basketball Features MAE Std: 122054.23
Basketball Features MAE: 3978935.4695
Basketball Features R2: 0.6426


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.



In [166]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import uniform, randint
import pandas as pd
import time
from scikeras.wrappers import KerasRegressor

np.random.seed(42)
tf.random.set_seed(42)

def create_cnn_model(learning_rate=0.001, l2_reg=0.01, dropout_rate_1=0.2, 
                     dropout_rate_2=0.2, dropout_rate_3=0.3, filters_1=32, 
                     filters_2=16, dense_units=50, input_dim=None):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters_1, kernel_size=3, activation='relu', 
                              input_shape=(input_dim, 1), padding='same'),
        tf.keras.layers.Dropout(dropout_rate_1),
        tf.keras.layers.Conv1D(filters=filters_2, kernel_size=3, activation='relu', 
                              padding='same'),
        tf.keras.layers.Dropout(dropout_rate_2),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(dense_units, activation='relu', 
                             kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.Dropout(dropout_rate_3),
        tf.keras.layers.Dense(1)  
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

results = []

def evaluate_model(model, X_train, X_test, y_train, y_test, feature_set):
    try:
        start_time = time.time()
        model.fit(X_train, y_train.values.ravel())
        train_time = time.time() - start_time

        y_test_pred = model.predict(X_test)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        mae_folds = []
        for train_idx, val_idx in cv.split(X_train):
            X_train_fold = X_train[train_idx]
            X_val_fold = X_train[val_idx]
            y_train_fold = y_train.iloc[train_idx] if isinstance(y_train, pd.DataFrame) else y_train[train_idx]
            y_val_fold = y_train.iloc[val_idx] if isinstance(y_train, pd.DataFrame) else y_train[val_idx]
            
            model.fit(X_train_fold, y_train_fold, epochs=50, batch_size=32, verbose=0)
            y_pred_fold = model.predict(X_val_fold, verbose=0).flatten()
            mae_folds.append(mean_absolute_error(y_val_fold, y_pred_fold))
        
        cv_mae_mean = np.mean(mae_folds)
        cv_mae_std = np.std(mae_folds, ddof=1)
        
        print(f"\nDiagnostics for 1D CNN ({feature_set}):")
        print(f"CV MAE Folds: {[f'{mae:.2f}' for mae in mae_folds]}")
        print(f"CV MAE Mean: {cv_mae_mean:.2f}")
        print(f"CV MAE Std: {cv_mae_std:.2f}")
        print(f"Training Time (s): {train_time:.4f}")

        mae_scale = 1_000_000
        results.append({
            'Model': '1D CNN Regression',
            'Feature Set': feature_set,
            'Dataset': 'Test',
            'MAE': test_mae,
            'R2': test_r2,
            'CV MAE Mean': cv_mae_mean,
            'CV MAE Std': cv_mae_std
        })
    except Exception as e:
        print(f"Error evaluating model for {feature_set}: {e}")
        results.append({
            'Model': '1D CNN Regression',
            'Feature Set': feature_set,
            'Dataset': 'Test',
            'MAE': np.nan,
            'R2': np.nan,
            'CV MAE Mean': np.nan,
            'CV MAE Std': np.nan
        })
    return model

param_dist = {
    'model__learning_rate': uniform(1e-5, 1e-3),  
    'model__l2_reg': uniform(0.01, 0.1),         
    'model__dropout_rate_1': uniform(0.2, 0.4),  
    'model__dropout_rate_2': uniform(0.2, 0.4), 
    'model__dropout_rate_3': uniform(0.3, 0.4),  
    'model__filters_1': randint(16, 64),        
    'model__filters_2': randint(8, 32),         
    'model__dense_units': randint(20, 100),     
    'batch_size': [8, 16, 32]
}

try:
    cnn_basketball = KerasRegressor(
        model=create_cnn_model,
        epochs=50,
        verbose=0,
        input_dim=X_train_scaled_cnn.shape[1]
    )
    grid_search_basketball = RandomizedSearchCV(cnn_basketball, param_dist, n_iter=10, cv=5, 
                                               scoring='r2', n_jobs=1, random_state=42, 
                                               error_score='raise')
    grid_search_basketball.fit(X_train_scaled_cnn, y_aug.values.ravel())
    cnn_basketball = evaluate_model(grid_search_basketball.best_estimator_, 
                                   X_train_scaled_cnn, X_test_scaled_cnn, 
                                   y_aug, y_test, 'Basketball Features')
    print(f"Best 1D CNN params (Basketball Features): {grid_search_basketball.best_params_}")
except Exception as e:
    print(f"Error in randomized search (Basketball Features): {e}")
    cnn_basketball = None

results_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.4f}'.format)  

print("\n1D CNN Regression Results:")
print(results_df)

Error in randomized search (Basketball Features): 'super' object has no attribute '__sklearn_tags__'

1D CNN Regression Results:
Empty DataFrame
Columns: []
Index: []


In [168]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import time
from scikeras.wrappers import KerasRegressor

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define the CNN model creation function
def create_cnn_model(learning_rate=0.001, l2_reg=0.01, dropout_rate_1=0.44, 
                     dropout_rate_2=0.2, dropout_rate_3=0.3, filters_1=42, 
                     filters_2=17, dense_units=95, input_dim=None):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters_1, kernel_size=3, activation='relu', 
                              input_shape=(input_dim, 1), padding='same'),
        tf.keras.layers.Dropout(dropout_rate_1),
        tf.keras.layers.Conv1D(filters=filters_2, kernel_size=3, activation='relu', 
                              padding='same'),
        tf.keras.layers.Dropout(dropout_rate_2),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(dense_units, activation='relu', 
                             kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.Dropout(dropout_rate_3),
        tf.keras.layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Initialize results list
results = []

# Define the evaluation function
def evaluate_model(model, X_train, X_test, y_train, y_test, feature_set):
    try:
        start_time = time.time()
        model.fit(X_train, y_train.values.ravel(), epochs=50, batch_size=8, verbose=0)
        train_time = time.time() - start_time

        y_test_pred = model.predict(X_test)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        mae_folds = []
        for train_idx, val_idx in cv.split(X_train):
            X_train_fold = X_train[train_idx]
            X_val_fold = X_train[val_idx]
            y_train_fold = y_train.iloc[train_idx] if isinstance(y_train, pd.DataFrame) else y_train[train_idx]
            y_val_fold = y_train.iloc[val_idx] if isinstance(y_train, pd.DataFrame) else y_train[val_idx]
            
            model.fit(X_train_fold, y_train_fold, epochs=50, batch_size=32, verbose=0)
            y_pred_fold = model.predict(X_val_fold, verbose=0).flatten()
            mae_folds.append(mean_absolute_error(y_val_fold, y_pred_fold))
        
        cv_mae_mean = np.mean(mae_folds)
        cv_mae_std = np.std(mae_folds, ddof=1)
        
        print(f"\nDiagnostics for 1D CNN ({feature_set}):")
        print(f"CV MAE Folds: {[f'{mae:.2f}' for mae in mae_folds]}")
        print(f"CV MAE Mean: {cv_mae_mean:.2f}")
        print(f"CV MAE Std: {cv_mae_std:.2f}")
        print(f"Training Time (s): {train_time:.4f}")

        results.append({
            'Model': '1D CNN Regression',
            'Feature Set': feature_set,
            'Dataset': 'Test',
            'MAE': test_mae,
            'R2': test_r2,
            'CV MAE Mean': cv_mae_mean,
            'CV MAE Std': cv_mae_std
        })
    except Exception as e:
        print(f"Error evaluating model for {feature_set}: {e}")
        results.append({
            'Model': '1D CNN Regression',
            'Feature Set': feature_set,
            'Dataset': 'Test',
            'MAE': np.nan,
            'R2': np.nan,
            'CV MAE Mean': np.nan,
            'CV MAE Std': np.nan
        })
    return model

# Train and evaluate the model without RandomizedSearchCV
try:
    # Create KerasRegressor instance
    cnn_basketball = KerasRegressor(
        model=create_cnn_model,
        epochs=50,
        batch_size=32,
        verbose=0,
        input_dim=X_train_scaled_cnn.shape[1]
    )
    
    # Evaluate the model directly
    cnn_basketball = evaluate_model(
        cnn_basketball,
        X_train_scaled_cnn,
        X_test_scaled_cnn,
        y_aug,
        y_test,
        'Basketball Features'
    )
except Exception as e:
    print(f"Error training model (Basketball Features): {e}")
    cnn_basketball = None

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.4f}'.format)
print("\n1D CNN Regression Results:")
print(results_df)

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/s


Diagnostics for 1D CNN (Basketball Features):
CV MAE Folds: ['7177881.00', '6871311.00', '7142454.50', '7285862.50', '7768103.00']
CV MAE Mean: 7249122.40
CV MAE Std: 327828.28
Training Time (s): 21.0219

1D CNN Regression Results:
               Model          Feature Set Dataset          MAE     R2  \
0  1D CNN Regression  Basketball Features    Test 6030550.5000 0.0225   

   CV MAE Mean  CV MAE Std  
0 7249122.4000 327828.2821  


In [205]:
def create_optimized_1d_cnn(input_dim = 28, output_dim=1, learning_rate=0.01, 
                            filters_1=16, filters_2=8, dense_units=50, 
                            dropout_rate_1=0.3, dropout_rate_2=0.3, dropout_rate_3=0.4):
    model = models.Sequential()
    
    # Fully connected layer to increase dimension
    model.add(layers.Dense(128, input_shape=(input_dim,), activation='relu'))
    
    # Reshape to (16, 8) for Conv1D input
    model.add(layers.Reshape((16, 8)))
    
    # First Conv1D layer with batch normalization and dropout
    model.add(layers.Conv1D(filters=filters_1, kernel_size=3, activation='leaky_relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_1))
    
    # Second Conv1D layer with batch normalization and dropout
    model.add(layers.Conv1D(filters=filters_2, kernel_size=3, activation='leaky_relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_2))
    
    # Flatten the output
    model.add(layers.Flatten())
    
    # Dense layer for regression
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate_3))
    
    # Output layer for regression
    model.add(layers.Dense(output_dim))
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    
    return model

In [206]:
model = create_optimized_1d_cnn(input_dim=X_train_scaled_cnn.shape[1])
model.fit(X_train_scaled_cnn, y_aug, epochs=500, batch_size=16, validation_split=0.2)

Epoch 1/500


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



368/368 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step - loss: 194228822999040.0000 - val_loss: 153982915837952.0000
Epoch 2/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 143531062591488.0000 - val_loss: 56392786378752.0000
Epoch 3/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 61652527480832.0000 - val_loss: 42179208675328.0000
Epoch 4/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 49758404083712.0000 - val_loss: 41456865640448.0000
Epoch 5/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 46674428821504.0000 - val_loss: 40496659431424.0000
Epoch 6/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 46801935663104.0000 - val_loss: 40307706036224.0000
Epoch 7/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 44698722893824.0000 - val_loss: 40426677469184.0000
Epoch 8/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - loss: 43743675678720.0000 - val_loss: 40006097829888.0000
Epoch 9/500
368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - loss: 45133240205312.0000 - val_

In [207]:
y_pred = model.predict(X_test_scaled_cnn)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"R²: {r2}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
MAE: 4011558.5
R²: 0.5960239171981812
